<a href="https://colab.research.google.com/github/kong1032002/BigCorp/blob/main/rugpull_classification_feature_token_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers as L
from tensorflow import keras
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
from datetime import datetime

Exception ignored in: <function _xla_gc_callback at 0x7cf116a39480>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


**Đọc dữ liệu trainning dataset và chuẩn hóa tên các cột thành lower case**

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Dataset_v1.7.csv")
df.dropna(inplace=True)
df.rename(columns=lambda x: x.lower(), inplace=True)
print(df.shape)
df.head()

(18348, 20)


,id,label,mint_count_per_week,burn_count_per_week,mint_ratio,swap_ratio,burn_ratio,mint_mean_period,swap_mean_period,burn_mean_period,swapin_per_week,swapout_per_week,swap_rate,lp_avg,lp_stdev,lpcreator_holding_ratio,lock_ratio,token_burn_ratio,creator_token_holding_ratio,number_of_token_creation_of_creator
0,0x79be722ac565ea6e0393b04604b766c6d6a07ed8,True,0.925083,0.0,0.013158,0.986842,0.0,0.0,0.105351,0.0,69.381244,0.000000,75.000000,100.0,0.0,0.999666,0.0,0.0,8.960000e+02,1
1,0x5e58b5210e069fd4dcea10d94e735b0809b6a6b3,True,0.906666,0.0,0.011765,0.988235,0.0,0.0,0.085498,0.0,76.159980,0.000000,84.000000,100.0,0.0,0.999522,0.0,0.0,9.999432e-10,1
2,0xf690325d046cc76b1f88c6d84b9ca7673f1b68ed,True,0.883027,0.0,0.021739,0.978261,0.0,0.0,0.084539,0.0,39.736202,0.000000,45.000000,100.0,0.0,0.999132,0.0,0.0,1.351351e-15,1
3,0x4014f7d9b0d0196da20199c9ad6c3e4251feb9fb,True,0.772323,0.0,0.007092,0.992908,0.0,0.0,0.163984,0.0,108.125227,0.000000,140.000000,100.0,0.0,0.999491,0.0,0.0,5.000000e+06,1
4,0x8b72aceb3e49a4c343ee538ff96b8497ba14575c,True,0.962898,0.0,0.040000,0.960000,0.0,0.0,0.098610,0.0,21.183753,1.925796,7.333333,100.0,0.0,1.000497,0.0,0.0,0.000000e+00,1


**Chuyển đổi label của trainning dataframe từ Boolean về Int**

In [ ]:
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])
print(df["label"].value_counts())
df.head()

1    16511
0     1837
Name: label, dtype: int64


,id,label,mint_count_per_week,burn_count_per_week,mint_ratio,swap_ratio,burn_ratio,mint_mean_period,swap_mean_period,burn_mean_period,swapin_per_week,swapout_per_week,swap_rate,lp_avg,lp_stdev,lpcreator_holding_ratio,lock_ratio,token_burn_ratio,creator_token_holding_ratio,number_of_token_creation_of_creator
0,0x79be722ac565ea6e0393b04604b766c6d6a07ed8,1,0.925083,0.0,0.013158,0.986842,0.0,0.0,0.105351,0.0,69.381244,0.000000,75.000000,100.0,0.0,0.999666,0.0,0.0,8.960000e+02,1
1,0x5e58b5210e069fd4dcea10d94e735b0809b6a6b3,1,0.906666,0.0,0.011765,0.988235,0.0,0.0,0.085498,0.0,76.159980,0.000000,84.000000,100.0,0.0,0.999522,0.0,0.0,9.999432e-10,1
2,0xf690325d046cc76b1f88c6d84b9ca7673f1b68ed,1,0.883027,0.0,0.021739,0.978261,0.0,0.0,0.084539,0.0,39.736202,0.000000,45.000000,100.0,0.0,0.999132,0.0,0.0,1.351351e-15,1
3,0x4014f7d9b0d0196da20199c9ad6c3e4251feb9fb,1,0.772323,0.0,0.007092,0.992908,0.0,0.0,0.163984,0.0,108.125227,0.000000,140.000000,100.0,0.0,0.999491,0.0,0.0,5.000000e+06,1
4,0x8b72aceb3e49a4c343ee538ff96b8497ba14575c,1,0.962898,0.0,0.040000,0.960000,0.0,0.0,0.098610,0.0,21.183753,1.925796,7.333333,100.0,0.0,1.000497,0.0,0.0,0.000000e+00,1


**Đọc dữ liệu test dataset và chuẩn hóa tên các cột thành lower case**

In [ ]:
test_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Dataset_v1.8.csv")
test_df.dropna(inplace=True)
test_df.rename(columns=lambda x: x.lower(), inplace=True)
print(test_df.shape)
test_df.head()

(18334, 20)


,id,label,mint_count_per_week,burn_count_per_week,mint_ratio,swap_ratio,burn_ratio,mint_mean_period,swap_mean_period,burn_mean_period,swapin_per_week,swapout_per_week,swap_rate,lp_avg,lp_stdev,lpcreator_holding_ratio,lock_ratio,token_burn_ratio,creator_token_holding_ratio,number_of_token_creation_of_creator
0,0xc45681eed9bea2a71cdcc1fa324a40f1d4617285,True,3.624183,0.0,0.666667,0.333333,0.0,0.267173,0.036865,0.0,1.812091,0.000000,2.000000,100.0,0.0,1.000000,0.000000,0.000000,1.160000e+66,2
1,0x624585c31d9c40cca7ef3151d863da150450bf41,True,0.070128,0.0,0.007143,0.992857,0.0,0.000000,0.340321,0.0,4.488208,5.259619,0.842105,100.0,0.0,0.000016,1.000607,0.000000,1.040000e+56,1
2,0x163c9e4df29498537b99e0f729e36e4626576a8b,True,0.979194,0.0,0.050000,0.950000,0.0,0.000000,0.183410,0.0,17.625488,0.979194,9.000000,100.0,0.0,0.000000,0.999937,0.314502,1.000000e+16,1
3,0x45db9c78e5751e06fabf1ed7559607b2c0896652,True,0.998964,0.0,0.125000,0.875000,0.0,0.000000,0.234450,0.0,3.995857,2.996893,1.000000,100.0,0.0,0.250156,0.750467,0.000000,8.333330e+12,1
4,0xcad88d721601a5b440a4398f3bbd288c5d383b32,True,0.994511,0.0,0.040000,0.960000,0.0,0.000000,0.294488,0.0,16.906689,6.961578,2.125000,100.0,0.0,0.000000,0.999808,0.000000,3.000000e+06,1


**Chuyển đổi label của trainning dataframe từ Boolean về Int**

In [ ]:
test_df["label"] = label_encoder.fit_transform(test_df["label"])
print(test_df["label"].value_counts())
test_df.head()

1    16498
0     1836
Name: label, dtype: int64


,id,label,mint_count_per_week,burn_count_per_week,mint_ratio,swap_ratio,burn_ratio,mint_mean_period,swap_mean_period,burn_mean_period,swapin_per_week,swapout_per_week,swap_rate,lp_avg,lp_stdev,lpcreator_holding_ratio,lock_ratio,token_burn_ratio,creator_token_holding_ratio,number_of_token_creation_of_creator
0,0xc45681eed9bea2a71cdcc1fa324a40f1d4617285,1,3.624183,0.0,0.666667,0.333333,0.0,0.267173,0.036865,0.0,1.812091,0.000000,2.000000,100.0,0.0,1.000000,0.000000,0.000000,1.160000e+66,2
1,0x624585c31d9c40cca7ef3151d863da150450bf41,1,0.070128,0.0,0.007143,0.992857,0.0,0.000000,0.340321,0.0,4.488208,5.259619,0.842105,100.0,0.0,0.000016,1.000607,0.000000,1.040000e+56,1
2,0x163c9e4df29498537b99e0f729e36e4626576a8b,1,0.979194,0.0,0.050000,0.950000,0.0,0.000000,0.183410,0.0,17.625488,0.979194,9.000000,100.0,0.0,0.000000,0.999937,0.314502,1.000000e+16,1
3,0x45db9c78e5751e06fabf1ed7559607b2c0896652,1,0.998964,0.0,0.125000,0.875000,0.0,0.000000,0.234450,0.0,3.995857,2.996893,1.000000,100.0,0.0,0.250156,0.750467,0.000000,8.333330e+12,1
4,0xcad88d721601a5b440a4398f3bbd288c5d383b32,1,0.994511,0.0,0.040000,0.960000,0.0,0.000000,0.294488,0.0,16.906689,6.961578,2.125000,100.0,0.0,0.000000,0.999808,0.000000,3.000000e+06,1


In [ ]:
X = df.drop(['label', 'id'], axis = 1)
X_test = test_df.drop(['label', 'id'], axis = 1)
y = pd.get_dummies(df['label'])
y_test = pd.get_dummies(test_df['label'])
y

NameError: ignored

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [ ]:
class DataConfig:
    def __init__(self, numeric_feature_names, categorical_features_with_vocabulary):
        self.NUMERIC_FEATURE_NAMES = numeric_feature_names
        self.CATEGORICAL_FEATURES_WITH_VOCABULARY = categorical_features_with_vocabulary
        self.CATEGORICAL_FEATURE_NAMES = list(self.CATEGORICAL_FEATURES_WITH_VOCABULARY.keys())
        self.FEATURE_NAMES = self.NUMERIC_FEATURE_NAMES + self.CATEGORICAL_FEATURE_NAMES

class DataLoader:
    @classmethod
    def from_df(cls, X, y=None, batch_size=1024):
        return (
            Dataset.from_tensor_slices(({col: X[col].values.tolist() for col in X.columns}, y.values.tolist())).batch(
                batch_size
            )
            if y is not None
            else Dataset.from_tensor_slices({col: X[col].values.tolist() for col in X.columns}).batch(batch_size)
        )

In [ ]:
def get_inputs(config):
    return {
        feature_name: L.Input(
            name=feature_name,
            shape=(),
            dtype=(tf.float32 if feature_name in config.NUMERIC_FEATURE_NAMES else tf.string),
        )
        for feature_name in config.FEATURE_NAMES
    }

def encode_inputs(inputs, config, use_embeddings=False, embedding_dim=32, prefix="", concat_features=False):
    cat_features = []
    num_features = []
    for feature_name in inputs:
        if feature_name in config.CATEGORICAL_FEATURE_NAMES:
            vocabulary = config.CATEGORICAL_FEATURES_WITH_VOCABULARY[feature_name]
            lookup = L.StringLookup(
                vocabulary=vocabulary,
                mask_token=None,
                num_oov_indices=0,
                output_mode="int" if use_embeddings else "binary",
                name=f"{prefix}{feature_name}_lookup",
            )
            if use_embeddings:
                encoded_feature = lookup(inputs[feature_name])
                embedding = L.Embedding(
                    input_dim=len(vocabulary),
                    output_dim=embedding_dim,
                    name=f"{prefix}{feature_name}_embeddings",
                )
                encoded_feature = embedding(encoded_feature)
            else:
                encoded_feature = lookup(
                    L.Reshape((1,), name=f"{prefix}{feature_name}_reshape")(inputs[feature_name])
                )
            cat_features.append(encoded_feature)
        else:
            encoded_feature = L.Reshape((1,), name=f"{prefix}{feature_name}_reshape")(inputs[feature_name])
            num_features.append(encoded_feature)

    features = (
        L.Concatenate(name=f"{prefix}inputs_concatenate")(cat_features + num_features)
        if concat_features
        else (cat_features, num_features)
    )

    return features

In [ ]:
class FeatureTokenizerTransformerConfig:
    def __init__(
        self,
        num_outputs,
        out_activation,
        num_transformer_blocks=2,
        num_heads=8,
        embedding_dim=32,
        dense_dim=16,
    ):
        self.NUM_OUT = num_outputs
        self.OUT_ACTIVATION = out_activation
        self.NUM_TRANSFORMER_BLOCKS = num_transformer_blocks
        self.NUM_HEADS = num_heads
        self.EMBEDDING_DIM = embedding_dim
        self.DENSE_DIM = dense_dim

In [ ]:
class TransformerBlock(L.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = L.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = tf.keras.Sequential([L.Dense(dense_dim, activation="relu"), L.Dense(embed_dim)])
        self.layernorm1 = L.LayerNormalization()
        self.layernorm2 = L.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[: tf.newaxis, :]
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm2(proj_input + proj_output)

In [ ]:
class FeatureTokenizerTransformer:
    @classmethod
    def from_config(cls, data_config, model_config, name):
        inputs = get_inputs(data_config)
        cat_features, num_features = encode_inputs(
            inputs,
            data_config,
            use_embeddings=True,
            embedding_dim=model_config.EMBEDDING_DIM,
            prefix="",
            concat_features=False,
        )
        num_feat_emb = [
            L.Dense(model_config.EMBEDDING_DIM, name=f"{feature_name}_embeddings")
            for _, feature_name in zip(range(len(num_features)), data_config.NUMERIC_FEATURE_NAMES)
        ]
        num_features = [emb(feat) for emb, feat in zip(num_feat_emb, num_features)]

        features = L.Concatenate(axis=1, name="feature_embeddings_stack")(
            [
                L.Reshape((1, 32), name=f"{feat_name}_reshape_2")(feat)
                for feat, feat_name in zip((num_features + cat_features), data_config.FEATURE_NAMES)
            ]
        )

        for _ in range(model_config.NUM_TRANSFORMER_BLOCKS):
            features = TransformerBlock(
                embed_dim=model_config.EMBEDDING_DIM,
                dense_dim=model_config.DENSE_DIM,
                num_heads=model_config.NUM_HEADS,
            )(features)
        features = L.GlobalMaxPooling1D()(features)
        outputs = L.Dense(
            units=model_config.NUM_OUT,
            activation=model_config.OUT_ACTIVATION,
            name="outputs",
        )(features)
        model = keras.Model(inputs=inputs, outputs=outputs, name=name)
        return model

In [ ]:
data_config = DataConfig(
    numeric_feature_names=X.columns.tolist(), categorical_features_with_vocabulary={}
)
model_config = FeatureTokenizerTransformerConfig(num_outputs=len(y.columns), out_activation='softmax')

In [ ]:
blank_model = FeatureTokenizerTransformer.from_config(data_config, model_config, name='ftt')
blank_model.summary()

Model: "ftt"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 mint_count_per_week (Input  [(None,)]                    0         []                            
 Layer)                                                                                           
                                                                                                  
 burn_count_per_week (Input  [(None,)]                    0         []                            
 Layer)                                                                                           
                                                                                                  
 mint_ratio (InputLayer)     [(None,)]                    0         []                            
                                                                                                

In [ ]:
MAX_EPOCHS  = 50

get_callbacks = lambda : [
    keras.callbacks.EarlyStopping(min_delta=1e-4, patience=3, verbose=1, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(patience=2, verbose=1)
]

In [ ]:
from tensorflow.data import Dataset
preds = []

for fold, (train_index, valid_index) in enumerate(skf.split(X, df['label'])):
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

    scaler = StandardScaler().fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
    X_valid = pd.DataFrame(scaler.transform(X_valid), columns=X.columns)
    x_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)

    data_train = DataLoader.from_df(X_train, y_train, batch_size=512)
    data_valid = DataLoader.from_df(X_valid, y_valid, batch_size=512)
    data_test = DataLoader.from_df(x_test, batch_size=512)

    model = FeatureTokenizerTransformer.from_config(data_config, model_config, name=f'ftt_fold_{fold}')
    model.compile(
        loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy']
    )
    model.fit(
        data_train, validation_data=data_valid, callbacks=get_callbacks(),
        epochs=MAX_EPOCHS
    )
    preds.append(model.predict(data_test))

Epoch 1/50
29/29 [==============================] - 15s 63ms/step - loss: 0.3077 - accuracy: 0.8888 - val_loss: 0.1668 - val_accuracy: 0.9387 - lr: 0.0010
Epoch 2/50
29/29 [==============================] - 1s 29ms/step - loss: 0.1682 - accuracy: 0.9418 - val_loss: 0.1408 - val_accuracy: 0.9343 - lr: 0.0010
Epoch 3/50
29/29 [==============================] - 1s 31ms/step - loss: 0.1970 - accuracy: 0.9227 - val_loss: 0.1359 - val_accuracy: 0.9531 - lr: 0.0010
Epoch 4/50
29/29 [==============================] - 1s 35ms/step - loss: 0.1864 - accuracy: 0.9358 - val_loss: 0.1339 - val_accuracy: 0.9392 - lr: 0.0010
Epoch 5/50
29/29 [==============================] - 1s 36ms/step - loss: 0.2071 - accuracy: 0.9248 - val_loss: 0.2737 - val_accuracy: 0.9368 - lr: 0.0010
Epoch 6/50
29/29 [==============================] - 1s 36ms/step - loss: 0.2329 - accuracy: 0.9255 - val_loss: 0.1270 - val_accuracy: 0.9545 - lr: 0.0010
Epoch 7/50
29/29 [==============================] - 1s 35ms/step - loss: 0.

In [ ]:
submissions = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')
submissions['target'] = pd.DataFrame(
    np.array([arr for arr in preds]).mean(axis=0),columns=y.columns
).idxmax(axis=1).values.tolist()
submissions.to_csv('preds.csv', index=False)